In [1]:
import re
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec
import random
import string
import math
from tqdm import tqdm
# matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Plot imports
import plotly.plotly as py
import plotly.graph_objs as go

## Offline mode
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline()
# Set the global theme
cf.set_config_file(world_readable=True, theme='pearl', offline=True)

a good post about match

https://stackoverflow.com/questions/24958358/use-regular-expression-in-python-to-find-two-strings-in-line

a table details the equivalence of ontology an classic effect description

http://snpeff.sourceforge.net/SnpEff_manual.html

# 1. parsing mutationseq vcf files

In [2]:
def parse_mutationseq_vcf(vcf):
    '''
    include all variants high, moderate, low, and modifiers
    '''
    df = pd.read_csv(vcf, comment='#', sep='\t', header=None, low_memory=False)
    patient = vcf.split('/')[4]
    df = df[[0,1,3,4,7]]
    df.columns = ['chr', 'pos', 'ref', 'alt', 'effect']
    df = df[(df['effect'].str.contains("HIGH"))|(df['effect'].str.contains("MODERATE"))|(df['effect'].str.contains("LOW"))|(df['effect'].str.contains("MODIFIER"))]
    if not df.empty:
        df['impact'], df['impact_type'], df['gene'] = df['effect'].apply(lambda x: parse_effect(x)).str.split('@').str
        df['patient'] = patient
    df = df.drop('effect', axis=1)
    return df

In [3]:
# keep it easy for now pick HIGH, MODERATE and then LOW
def parse_effect(line):
    effs = line.split('EFF=')[1].split(',')
    #     extract impact, impact_type and gene
    effs = ['@'.join(list(np.array(re.split('\(|\|',ef))[[0,1,6]])) for ef in effs if ('HIGH' in  ef) or ('MODERATE' in ef) or ('LOW' in ef) or ('MODIFIER' in ef)]
    effs = list(set(effs))
    high = [ef for ef in effs if 'HIGH' in ef]
    moderate = [ef for ef in effs if 'MODERATE' in ef]
    low = [ef for ef in effs if 'LOW' in ef]
    modifier = [ef for ef in effs if 'MODIFIER' in ef]
    if high:
        anno = high[0]
    elif moderate:
        anno = moderate[0]
    elif low:
        anno = low[0]
    elif modifier:
        anno = modifier[0]
    else:
        print('ERROR!')
#     make sure the genes have the same name
    genes = [ef.split('@')[2] for ef in effs]
    
    return anno


In [4]:
# # example run
# ft = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02020/hg19a/GSH/A37247_A56468/strelka/23223/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf'
# # assert count_coding_mutations(ft) == 266
# df = parse_strelka_vcf(ft)

# 2. Scope: genes and cohort size

In [4]:
# 118 patients
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
with open(f) as file:
    patients = [line.strip() for line in file]
assert len(patients) == 118

In [3]:
# 12 SMGS
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed.txt'
with open (f2) as file:
    genes = [line.strip() for line in file]
assert len(genes) == 12

# 3. mutationseq vcf files

In [5]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf_20190115.txt'
df = pd.read_csv(f1, sep='\t', index_col='patient')
df.head(2)

df.columns

ndf = df['mutseq_snv_vcf'].dropna()
ndf.head(2)

ndf = ndf.reindex(patients)
ndf.shape
ndf.loc['HTMCP-03-06-02261',]

,HIV_status,DNA_lib,source,status,RNA_lib,ribodepleted_lib,DNA_bam,RNA_bam,DNA_single_vcf,paired_mpileup_vcf,...,other_vcf,DNA_tc,RNA_tc,cnv,bbt_transcriptome,bbt_genome,bbt_transcriptome_other_bacterial,bbt_genome_other_bacterial,bbt_transcriptome_other_viral,bbt_genome_other_viral
patient,,,,,,,,,,,,,,,,,,,,,
HTMCP-03-06-02001,Positive,A37234,HTMCP_124,Malignant,A37700,A56295,/projects/analysis/analysis22/A37234/merge_bwa...,/projects/analysis/analysis22/IX3433/C67GDANXX...,/projects/analysis/analysis22/A37234/merge_bwa...,NaN,...,NaN,55.0,55.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...
HTMCP-03-06-02002,Negative,A37235,HTMCP_125,Malignant,A37701,A56296,/projects/analysis/analysis22/A37235/merge_bwa...,/projects/analysis/analysis22/IX3432/C67GDANXX...,/projects/analysis/analysis22/A37235/merge_bwa...,NaN,...,NaN,70.0,70.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...


Index(['HIV_status', 'DNA_lib', 'source', 'status', 'RNA_lib',
       'ribodepleted_lib', 'DNA_bam', 'RNA_bam', 'DNA_single_vcf',
       'paired_mpileup_vcf', 'strelka_snv_vcf', 'strelka_indel_vcf',
       'mutseq_snv_vcf', 'mutseq_snv_vcf_old', 'RNA_single_vcf', 'other_vcf',
       'DNA_tc', 'RNA_tc', 'cnv', 'bbt_transcriptome', 'bbt_genome',
       'bbt_transcriptome_other_bacterial', 'bbt_genome_other_bacterial',
       'bbt_transcriptome_other_viral', 'bbt_genome_other_viral'],
      dtype='object')

patient
HTMCP-03-06-02001    /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02002    /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
Name: mutseq_snv_vcf, dtype: object

(118,)

'/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02261/hg19a/GSH/A68235_A66443/mutationseq/63195/A68235_A66443.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf'

# 4. concatenate snvs, mutationseq does not report indels

## look into ~800 pog strelka snv calls to see if systematic noise exist

In [22]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/pogs/pog_strelka_snv_calls.txt'
df1 = pd.read_csv(f, header=None)[0].tolist()
df1[:2]
dfmg = pd.DataFrame()
for p in df1:
    patient = p.split('/')[4]
    print(f'working on {patient}')
    snv_df = parse_mutationseq_vcf(p)
    mdf = snv_df
    if dfmg.empty:
        dfmg = mdf
    else:
        dfmg = pd.concat([dfmg, mdf], sort=False)

['/projects/POG/POG_data/POG001/wgs/arch1_t_A10509_blood1_n_A10501/strelka/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf',
 '/projects/POG/POG_data/POG001/wgs/biop1_t_A10504_blood1_n_A10501/strelka/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf']

working on POG001
working on POG001
working on POG002
working on POG002
working on POG003
working on POG003
working on POG003
working on POG004
working on POG005
working on POG005
working on POG006
working on POG007
working on POG008
working on POG008
working on POG009
working on POG010
working on POG011
working on POG011
working on POG012
working on POG012
working on POG012
working on POG014
working on POG015
working on POG015
working on POG016
working on POG017
working on POG017
working on POG018
working on POG018
working on POG019
working on POG019
working on POG020
working on POG020
working on POG021
working on POG021
working on POG021
working on POG022
working on POG022
working on POG023
working on POG023
working on POG024
working on POG025
working on POG025
working on POG026
working on POG026
working on POG027
working on POG027
working on POG029
working on POG030
working on POG030
working on POG031
working on POG032
working on POG033
working on POG034
working on POG034
working on

working on POG460
working on POG461
working on POG462
working on POG463
working on POG465
working on POG468
working on POG469
working on POG469
working on POG470
working on POG472
working on POG473
working on POG474
working on POG475
working on POG476
working on POG477
working on POG478
working on POG479
working on POG480
working on POG482
working on POG485
working on POG486
working on POG487
working on POG487
working on POG488
working on POG489
working on POG490
working on POG491
working on POG495
working on POG495
working on POG496
working on POG497
working on POG498
working on POG499
working on POG502
working on POG505
working on POG506
working on POG507
working on POG509
working on POG511
working on POG512
working on POG513
working on POG514
working on POG515
working on POG517
working on POG518
working on POG519
working on POG521
working on POG524
working on POG526
working on POG530
working on POG531
working on POG532
working on POG533
working on POG534
working on POG539
working on

EmptyDataError: No columns to parse from file

In [24]:
dfmg.shape
dfmg.head()
# dfmg.patient.nunique() # 696 pogs run through at this time

(24817057, 8)

,chr,pos,ref,alt,impact,impact_type,gene,patient
0,1,825972,G,A,INTERGENIC,MODIFIER,,POG001
1,1,1063055,G,A,INTERGENIC,MODIFIER,,POG001
2,1,1277672,C,T,INTRON,MODIFIER,DVL1,POG001
3,1,1922492,G,C,INTRON,MODIFIER,C1orf222,POG001
4,1,2130700,A,C,EXON,MODIFIER,C1orf86,POG001


696

In [25]:
snv_counts = dfmg.groupby(['chr', 'pos'])['patient'].nunique().sort_values(ascending=False)

In [32]:
snv_counts[:20]

chr         pos     
12          25398284    70
GL000195.1  7094        60
X           58562237    60
1           565286      57
X           58560599    57
GL000205.1  109615      56
GL000220.1  64524       56
GL000195.1  7594        56
MT          72          55
15          84054881    55
GL000205.1  109647      53
8           43794763    53
GL000220.1  65083       53
GL000214.1  20627       52
GL000195.1  61438       52
8           43824536    51
GL000220.1  65708       51
GL000195.1  7559        51
GL000220.1  85672       51
GL000205.1  101824      51
Name: patient, dtype: int64

In [37]:
df11 = pd.DataFrame(snv_counts[snv_counts>14]).reset_index()

In [41]:
f11 = '/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/pogs/pog_strelka_snv_calls_696_pogs_3_or_more.txt'
df11.to_csv(f11, sep='\t', index=False )

In [30]:
type(snv_counts)

pandas.core.series.Series

In [31]:
snv_counts.iplot(kind='hist',
                xTitle='#patients',
                yTitle='hist#patients',
                title='recurrent snvs of 696 pogs')

In [6]:
dfmg = pd.DataFrame()
for ix, row in pd.DataFrame(ndf).iterrows():
    patient = ix
    print(f'working on {patient}')
    snv_vcf = row['mutseq_snv_vcf']
#     indel_vcf = row['strelka_indel_vcf']
    snv_df = parse_mutationseq_vcf(snv_vcf)
#     indel_df = parse_strelka_vcf(indel_vcf)
#     mdf = pd.concat([snv_df, indel_df], sort=False)
#     mdf = mdf.drop_duplicates(keep='first')
    mdf = snv_df
    if dfmg.empty:
        dfmg = mdf
    else:
        dfmg = pd.concat([dfmg, mdf], sort=False)


working on HTMCP-03-06-02001
working on HTMCP-03-06-02002
working on HTMCP-03-06-02003
working on HTMCP-03-06-02006
working on HTMCP-03-06-02008
working on HTMCP-03-06-02012
working on HTMCP-03-06-02013
working on HTMCP-03-06-02020
working on HTMCP-03-06-02036
working on HTMCP-03-06-02037
working on HTMCP-03-06-02040
working on HTMCP-03-06-02042
working on HTMCP-03-06-02046
working on HTMCP-03-06-02047
working on HTMCP-03-06-02054
working on HTMCP-03-06-02057
working on HTMCP-03-06-02058
working on HTMCP-03-06-02063
working on HTMCP-03-06-02068
working on HTMCP-03-06-02070
working on HTMCP-03-06-02071
working on HTMCP-03-06-02074
working on HTMCP-03-06-02075
working on HTMCP-03-06-02076
working on HTMCP-03-06-02089
working on HTMCP-03-06-02092
working on HTMCP-03-06-02097
working on HTMCP-03-06-02098
working on HTMCP-03-06-02099
working on HTMCP-03-06-02103
working on HTMCP-03-06-02108
working on HTMCP-03-06-02109
working on HTMCP-03-06-02110
working on HTMCP-03-06-02113
working on HTM

In [7]:
dfmg.head()
dfmg.shape
dfmg.impact_type.unique()

,chr,pos,ref,alt,impact,impact_type,gene,patient
0,1,54586,T,C,INTERGENIC,MODIFIER,,HTMCP-03-06-02001
1,1,59734,A,G,UPSTREAM,MODIFIER,OR4G11P,HTMCP-03-06-02001
2,1,66327,A,T,DOWNSTREAM,MODIFIER,OR4G11P,HTMCP-03-06-02001
3,1,99072,C,T,INTRON,MODIFIER,RP11-34P13.7,HTMCP-03-06-02001
4,1,238990,T,A,UPSTREAM,MODIFIER,AP006222.2,HTMCP-03-06-02001


(7738105, 8)

array(['MODIFIER', 'LOW', 'HIGH', 'MODERATE'], dtype=object)

In [8]:
snv_counts = dfmg.groupby(['chr', 'pos'])['patient'].nunique().sort_values(ascending=False)

In [10]:
(snv_counts>=3).sum()

560480

In [11]:
snv_counts[snv_counts>=3].to_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/rainstorm/118_cervical_mutationseq_snvs_3_patients.txt', sep='\t')

In [94]:
# subset 12 genes of interest
dfs = dfmg[dfmg.gene.isin(genes)&dfmg.patient.isin(patients)]
dfs.head(2)
dfs.shape

,chr,pos,ref,alt,impact,impact_type,gene,patient
5769,3,178936091,G,A,NON_SYNONYMOUS_CODING,MODERATE,PIK3CA,HTMCP-03-06-02001
5771,3,178952117,A,T,NON_SYNONYMOUS_CODING,MODERATE,PIK3CA,HTMCP-03-06-02001


(168, 8)

In [95]:
dfs = dfs[['gene', 'impact', 'patient']]
dfs.shape

dfs['impact_new'] = dfs['impact'].apply(lambda x: x.upper())
dfs.drop('impact', axis=1, inplace=True)
dfs.drop_duplicates(keep='first', inplace=True)
dfs.shape

(168, 3)

(148, 3)

In [96]:
dfs['impact_new'].unique()

array(['NON_SYNONYMOUS_CODING', 'FRAME_SHIFT', 'STOP_GAINED',
       'MISSENSE_VARIANT', 'SYNONYMOUS_VARIANT', 'SYNONYMOUS_CODING',
       'SPLICE_SITE_ACCEPTOR', 'START_GAINED', 'CODON_DELETION',
       'CODON_CHANGE_PLUS_CODON_DELETION'], dtype=object)

In [97]:
dfs = dfs.drop_duplicates().groupby(['gene', 'patient']).agg({'impact_new': ','.join}).reset_index()
dfs.head()

,gene,patient,impact_new
0,CASP8,HTMCP-03-06-02001,FRAME_SHIFT
1,CASP8,HTMCP-03-06-02012,STOP_GAINED
2,CASP8,HTMCP-03-06-02036,NON_SYNONYMOUS_CODING
3,CASP8,HTMCP-03-06-02239,MISSENSE_VARIANT
4,CASP8,HTMCP-03-06-02260,FRAME_SHIFT


In [98]:
def impact_type(x):
#     print(x)
    xsplit = list(set(x.split(',')))
    type = xsplit[0].upper()
    if len(xsplit) > 1:
        impact = 'Multiple'
    elif len(xsplit) == 1:
         if '+' in xsplit[0]:
             impact = 'Multiple'
         elif type == 'MISSENSE_VARIANT' or type == 'NON_SYNONYMOUS_CODING':
             impact = 'Non-synonymous'
         elif type == 'SYNONYMOUS_VARIANT' or type == 'SYNONYMOUS_CODING':
             impact = 'Synonymous'
         elif  type == 'SPLICE_SITE_ACCEPTOR' or  type == 'SPLICE_SITE_DONOR':
             impact = 'Splite site'
         elif type == 'STOP_LOST':
             impact = 'Stop lost'
         elif type == 'STOP_GAINED':
             impact = 'Stop gained'
         elif  type == 'START_LOST':
             impact = 'Start lost'
         elif type == 'START_GAINED':
             impact = 'Start gained'
         elif type == 'FRAME_SHIFT' or type == 'FRAMESHIFT_VARIANT':
             impact = 'Frameshift'
         elif type == 'INFRAME_DELETION' or type == 'CODON_INSERTION' or type == 'CODON_DELETION' or type == 'CODON_CHANGE_PLUS_CODON_INSERTION' or type == 'CODON_CHANGE_PLUS_CODON_DELETION' or type.lower() == 'disruptive_inframe_deletion':
             impact = 'Codon indel'                
         else: exit(1);print('ERROR');print(x)
    return impact

In [99]:
dfs['impact'] = dfs['impact_new'].apply(lambda x: impact_type(x))
dfs = dfs[['gene', 'patient', 'impact']].drop_duplicates()
dfs = dfs.set_index(['gene', 'patient'])['impact'].unstack()
dfs.head(2)
dfs.shape

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,HTMCP-03-06-02047,...,HTMCP-03-06-02344,HTMCP-03-06-02346,HTMCP-03-06-02354,HTMCP-03-06-02392,HTMCP-03-06-02411,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02435,HTMCP-03-06-02441,HTMCP-03-06-02447
gene,,,,,,,,,,,,,,,,,,,,,
CASP8,Frameshift,NaN,NaN,Stop gained,NaN,NaN,Non-synonymous,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAT1,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,Synonymous,NaN,NaN,...,NaN,NaN,Stop gained,NaN,NaN,Frameshift,Multiple,Non-synonymous,NaN,NaN


(12, 80)

# 5. add in covariates

In [100]:
# get meta track for example histology
f3 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
ddf = pd.read_csv(f3, sep='\t')
ddf = ddf[['patient', 'HIV_status', 'Putative_histology']]
edf = ddf.set_index('patient').T
edf.loc['Putative_histology',].unique()
edf = edf[patients]
edf.head(2)
edf.shape

array(['Squamous', 'Adenosquamous', 'Adeno', 'Neuroendocrine'],
      dtype=object)

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous


(2, 118)

In [101]:
dfs = pd.concat([edf, dfs], sort=False)

In [102]:
dfs.shape

(14, 118)

In [103]:
dfs = dfs[patients]

# 6. add in mutation rate (fraction of patients)

In [104]:
dfs['occurrence'] = (dfs.notnull().sum(axis=1)/(dfs.shape[1] -2))
dfs['percentage'] = ['{0}({1}%)'.format(i[0], int(round(i[1]*100))) for i in zip(dfs.index.tolist(), dfs.occurrence)]
# dfs.loc['HIV_status', 'percentage'] = 'HIV_Status'
# dfs.loc['Putative_histology', 'percentage' ] = 'Puatative_histology'
sdf = dfs.drop('occurrence', axis=1).set_index('percentage', drop=True)

In [105]:
sdf

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
percentage,,,,,,,,,,,,,,,,,,,,,
HIV_status(102%),Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology(102%),Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous
CASP8(7%),Frameshift,NaN,NaN,NaN,NaN,Stop gained,NaN,NaN,Non-synonymous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAT1(19%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,NaN,...,NaN,NaN,Frameshift,Multiple,NaN,Non-synonymous,NaN,NaN,NaN,NaN
FBXW7(10%),NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Non-synonymous,NaN,Frameshift,NaN,NaN,NaN,NaN
MAPK1(5%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN
MLL2(15%),NaN,Stop gained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Frameshift,NaN,NaN,NaN,NaN
PCDHA9(3%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCDHGA12(5%),NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN


In [106]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed_details.txt'
sdf.to_csv(of, sep='\t')